In [11]:
import onnx
from io import BytesIO
from urllib import request
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms

from PIL import Image

In [4]:

# 1. Load the model
# The loader will automatically handle 'hair_classifier_v1.onnx.data'
# if it is in the same folder.
model_path = "/content/hair_classifier_v1.onnx"
model = onnx.load(model_path)

# 2. Extract and print Input nodes
print(f"{'='*10} INPUT NODES {'='*10}")
for input_node in model.graph.input:
    print(f"Name:  {input_node.name}")
    # Printing shape is often helpful to ensure your input data matches
    shape = [d.dim_value for d in input_node.type.tensor_type.shape.dim]
    print(f"Shape: {shape}\n")

# 3. Extract and print Output nodes
print(f"{'='*10} OUTPUT NODES {'='*10}")
for output_node in model.graph.output:
    print(f"Name:  {output_node.name}")
    shape = [d.dim_value for d in output_node.type.tensor_type.shape.dim]
    print(f"Shape: {shape}\n")
    print(output.name)

========== INPUT NODES ==========
Name:  input
Shape: [0, 3, 200, 200]

========== OUTPUT NODES ==========
Name:  output
Shape: [0, 1]

output


In [15]:
# 1. Load the uploaded image
img = Image.open(image_path)

# 2. Resize to 200x200
# Image.LANCZOS is a high-quality downsampling filter
img_resized = img.resize((200, 200), Image.Resampling.LANCZOS)

# 3. Save or View
img_resized.save('target_size_image.jpg')
img_resized.show()

print(f"New size: {img_resized.size}")

New size: (200, 200)


In [17]:
def get_first_pixel_value(image_path):
    # 1. Load the Image
    # PIL loads as RGB by default
    img = Image.open(image_path).convert('RGB')

    # 2. Resize to (200, 200)
    # We use Bilinear interpolation to match PyTorch's default behavior
    img = img.resize((200, 200), resample=Image.BILINEAR)

    # 3. Convert to Numpy Array and Scale to [0, 1]
    # Current Shape: (200, 200, 3) -> (Height, Width, Channels)
    img_np = np.array(img, dtype=np.float32) / 255.0

    # 4. Transpose to Channels First format (PyTorch Requirement)
    # New Shape: (3, 200, 200) -> (Channels, Height, Width)
    img_chw = np.transpose(img_np, (2, 0, 1))

    # 5. Normalize with ImageNet Statistics
    # Mean and Std must be reshaped to (3, 1, 1) to broadcast over the height and width
    mean = np.array([0.485, 0.456, 0.406]).reshape(3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(3, 1, 1)

    img_normalized = (img_chw - mean) / std

    # 6. Get the value of the first pixel in the R channel
    # Channel 0 = Red
    # Row 0 = Top
    # Col 0 = Left
    r_pixel_value = img_normalized[0, 0, 0]

    return r_pixel_value

# Usage
try:
    val = get_first_pixel_value(image_path)
    print(f"Value of the first pixel (R channel): {val}")
except FileNotFoundError:
    print("Error: Image file not found.")

Value of the first pixel (R channel): -1.0561691907816058


In [21]:
# 1. Define the Model Architecture (Same as before)
class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# 2. Instantiate Model
model = HairClassifier()
model.eval() # Set to evaluation mode

# 3. Pre-process the Image
img = Image.open(image_path)

preprocess = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Add batch dimension (1, 3, 200, 200)
input_tensor = preprocess(img).unsqueeze(0)

# 4. Forward Pass
with torch.no_grad():
    output = model(input_tensor)
    prediction = output.item()

print(f"Raw Model Output (Probability): {prediction:.4f}")
print("Interpretation:")
print(f"  If > 0.5: Model predicts Class 1 (Straight)")
print(f"  If < 0.5: Model predicts Class 0 (Curly)")

Raw Model Output (Probability): 0.4916
Interpretation:
  If > 0.5: Model predicts Class 1 (Straight)
  If < 0.5: Model predicts Class 0 (Curly)
